In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline

In [3]:
import os
import sys

MODULE_PATH = os.path.abspath(os.path.join("..", "src"))

if MODULE_PATH not in sys.path:
    sys.path.append(MODULE_PATH)

In [ ]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from natsort import natsorted

from patchify import patchify

import scipy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

from config import PATCH_SIZE, PATCH_STEP, IMAGE_CHANNELS

2025-05-23 14:28:11.616 | INFO     | config:<module>:11 - PROJ_ROOT path is: /home/code-dragon-03/Desktop/High Priority/Research Work/Learned-Image-Compression


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f'Using GPU: {device}')
else:   
    device = torch.device("cpu")
    print("Using CPU")  

In [ ]:
class KodakDataset(Dataset):
    def __init__(self, root, transform=None, patches=False):
        self.root = root
        self.transform = transform
        self.patches = patches
        self.images = natsorted(os.listdir(root))
        self.images = [os.path.join(root, img) for img in self.images if img.endswith('.png') or img.endswith('.jpg')]

    def __len__(self):        
        return len(self.images)

    def __getitem__(self, idx):
        IMG_PATH = self.images[idx]
        image = Image.open(IMG_PATH)
        if self.transform:
            image = self.transform(image)
        if self.patches:
            img = np.array(image)
            if img.ndim == 2:
                img = img[..., np.newaxis]
            patches = patchify(img, (PATCH_SIZE, PATCH_SIZE, IMAGE_CHANNELS), step=PATCH_STEP)
            return image, patches
        return image